## Sectoral Analysis
- https://readmedium.com/en/https:/medium.datadriveninvestor.com/financial-market-dynamics-made-simple-one-visualization-to-rule-them-all-f154ab316ca5

In [6]:
import os
import pandas as pd
import yfinance as yf

INPUT_CSV_PATH = '../data/updated_niftymidcap50_stocks.csv'  # Replace with the actual path to your CSV file

def get_ticker_data(tickers: list):
    '''
    Obtain the price data for all tickers specified. The outcome is a csv file
    of price data for each ticker in the data folder.
    
    Parameters
    ----------
    tickers : list
        A list of the tickers to download the data for
    '''
        
    data = yf.download(
        tickers=tickers,
        interval='1d',
        group_by='ticker',
        threads=True,
    )
    
    data = data.T
    all_downloaded_tickers = list(set([idx[0] for idx in data.index]))
    
    for ticker in all_downloaded_tickers:

        df = data.loc[ticker, :].T.sort_index().dropna()
        df.to_csv(f'data/{ticker}.csv', index=True)
            
    return


def read_tickers_from_csv(csv_path: str, column_name: str = 'Symbol') -> list:
    '''
    Read tickers from an input CSV file.
    
    Parameters
    ----------
    csv_path : str
        The path to the CSV file.
    column_name : str, optional
        The name of the column containing tickers, by default 'Symbol'.
        
    Returns
    -------
    list
        A list of tickers.
    '''
    return pd.read_csv(csv_path)[column_name].tolist()


if __name__ == '__main__':
    
    # Check if a directory exists called 'data', if not, create it
    if not os.path.isdir('data'):
        os.mkdir('data')
    
    # Read tickers from the input CSV file
    input_tickers = read_tickers_from_csv(INPUT_CSV_PATH)
    
    # Download one CSV file per ticker and place it in the data directory
    get_ticker_data(input_tickers)


[*********************100%%**********************]  50 of 50 completed

46 Failed downloads:
['SRF', 'CDSL', 'LODHA', 'MAHABANK', 'ABB', 'MAHLIFE', 'STARHEALTH', 'BLUEDART', 'AVANTIFEED', 'RALLIS', 'DIVISLAB', 'GLAND', 'MANAPPURAM', 'AMBUJACEM', 'CARBORUNIV', 'LEMONTREE', 'FINCABLES', 'BALKRISIND', 'DCMSHRIRAM', 'ASIANPAINT', 'BIRLACORPN', 'TATACOMM', 'LXCHEM', 'CANBK', 'TV18BRDCST', 'IRCTC', 'FDC', 'DIXON', 'GODREJCP', 'MPHASIS', 'SKFINDIA', 'VGUARD', 'JUBLPHARMA', 'NATCOPHARM', 'CUB', 'L&TFH', 'NUVOCO', 'PIIND', 'PVRINOX', 'TRENT', 'TATAINVEST', 'RAYMOND', 'PETRONET', 'HAVELLS', 'CLEAN']: Exception('%ticker%: No timezone found, symbol may be delisted')
['FSL']: Exception('%ticker%: No price data found, symbol may be delisted (1d 1925-03-26 -> 2024-03-02)')
[**************************************************220%%***************************************************]  110 of 50 completedFailed to get ticker 'CIPLA' reason: HTTPSConnectionPool(host='www.yahoo.com', port=443): Read timed o

In [3]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'browser'

INPUT_CSV_FILE = '../data/ind_nifty500list.csv'  # Replace with the actual path to your CSV file
DATA_DIR = 'data/'
CALC_DAYS = 20  # The number of days to calculate the percentage gain from

if __name__ == '__main__':
    
    # Read ticker symbols and industry information from the input CSV file
    input_data = pd.read_csv(INPUT_CSV_FILE)
    
    # Collect the growth statistics for each ticker in the list
    stats = []
    for ticker in input_data['Symbol'].tolist():
        
        try:
            df = pd.read_csv(DATA_DIR + ticker + '.csv')
            prices = df['Close'].values
            
            stats.append({
                'ticker': ticker,
                'growth': 100 * (prices[-1] / prices[-CALC_DAYS] - 1)
            })
        except Exception as e:
            print(ticker, ':', e)
        
    # Form into a dataframe so that we can add the industry information to it, 
    # and then aggregate to get the visualisation heatmap
    stats = (
        pd.DataFrame(stats)
        .merge(
            input_data[['Symbol', 'Industry']],
            left_on='ticker',
            right_on='Symbol',
            how='left',
        )
        .drop(columns='Symbol')
    )
    
    # To aggregate the metrics, we need to add flags to see if they are
    # satisfied, these are then counted in the following steps
    stats['positive'] = np.where(stats['growth'] > 0, 1, 0)
    stats['greater_than_5'] = np.where(stats['growth'] > 5, 1, 0)
    stats['greater_than_10'] = np.where(stats['growth'] > 10, 1, 0)
    stats['less_than_5'] = np.where(stats['growth'] < -5, 1, 0)
    stats['less_than_10'] = np.where(stats['growth'] < -10, 1, 0)
    
    # Get the full statistics, mapping from display : col name
    stats_to_show = {
        'Positive': 'positive',
        'Greater than 5% growth': 'greater_than_5',
        'Greater than 10% growth': 'greater_than_10',
        'Less than -5%': 'less_than_5',
        'Less than -10%': 'less_than_10',
    }
    
    # Perform the aggregation
    stats = stats.groupby('Industry').agg({
        col: 'mean' for col in stats_to_show.values()   
    })
    
    # Format the dataframe for display purposes
    stats = stats.rename(columns={v: k for k, v in stats_to_show.items()})
    stats = 100 * stats.round(2)
    
    # Produce and show the plotly figure
    fig = px.imshow(
        stats.T, 
        x=stats.index, 
        y=stats.columns,
        text_auto=True,
        color_continuous_scale='RdYlGn',
        zmin=0,
        zmax=100,
    )
    
    fig.update_layout(
        xaxis={'title': 'Industry'},
        yaxis={'title': 'Metric Type'},
        coloraxis={'colorbar': {'title': 'Amount (%)'}},
        legend={'x': 0, 'y': -0.1, 'orientation': 'h'},
        margin={'l': 50, 'r': 50, 'b': 50, 't': 25},
        width=1000,
        height=600,
    )
    
    fig.show()


360ONE : [Errno 2] No such file or directory: 'data/360ONE.csv'
3MINDIA : [Errno 2] No such file or directory: 'data/3MINDIA.csv'
ABB : [Errno 2] No such file or directory: 'data/ABB.csv'
ACC : [Errno 2] No such file or directory: 'data/ACC.csv'
AIAENG : [Errno 2] No such file or directory: 'data/AIAENG.csv'
APLAPOLLO : [Errno 2] No such file or directory: 'data/APLAPOLLO.csv'
AUBANK : [Errno 2] No such file or directory: 'data/AUBANK.csv'
AARTIDRUGS : [Errno 2] No such file or directory: 'data/AARTIDRUGS.csv'
AARTIIND : [Errno 2] No such file or directory: 'data/AARTIIND.csv'
AAVAS : [Errno 2] No such file or directory: 'data/AAVAS.csv'
ABBOTINDIA : [Errno 2] No such file or directory: 'data/ABBOTINDIA.csv'
ADANIENSOL : [Errno 2] No such file or directory: 'data/ADANIENSOL.csv'
ADANIENT : [Errno 2] No such file or directory: 'data/ADANIENT.csv'
ADANIGREEN : [Errno 2] No such file or directory: 'data/ADANIGREEN.csv'
ADANIPORTS : [Errno 2] No such file or directory: 'data/ADANIPORTS.csv

KeyError: 'ticker'